# Booking Agent
For powerful online LLMs such as GPT-4 or Claude, the FunctionCallingAgentWorker combined with tools with direct returns can perform the tasks of a simple booking agent. However, FunctionCallingAgentWorker does not yet support local LLMs. To address this, a workaround using ReActAgent is demonstrated in this notebook.

In [ ]:
# !pip install llama-index-core llama-index-llms-anthropic
import os

os.environ["ANTHROPIC_API_KEY"] = "sk-ant-..."

In [81]:
from typing import Optional

from llama_index.core.tools import FunctionTool
from llama_index.core.bridge.pydantic import BaseModel, Field
# we will store booking under random IDs
bookings = {}


# we will represent and track the state of a booking as a Pydantic model
class Booking(BaseModel):
    name: Optional[str] = None
    email: Optional[str] = None
    phone: Optional[str] = None
    date: Optional[str] = None
    time: Optional[str] = None


# def get_booking_state(user_id: str) -> str:
#     """Get the current state of a booking for a given booking ID."""
#     try:
#         return str(bookings[user_id].dict())
#     except:
#         return f"Booking ID {user_id} not found"

def get_booking_state(user_id: str=Field(
        description="user's id"
    )) -> str:
    """Get the current state of a booking for a given booking ID."""
    try:
        return str(bookings[user_id].dict())
    except:
        return f"Booking ID {user_id} not found"


# def update_booking(user_id: str, property: str, value: str) -> str:
#     """Update a property of a booking for a given booking ID. Only enter details that are explicitly provided."""
#     booking = bookings[user_id]
#     setattr(booking, property, value)
#     return f"Booking ID {user_id} updated with {property} = {value}"

def update_booking(user_id: str=Field(description="user's id"), 
        property: str=Field(description="property of a booking, should be one of the following: name, email, phone, date, time"), 
        value: str=Field(description="the value of the property")
        ) -> str:
    """Update a property of a booking for a given booking ID. Only enter details that are explicitly provided."""
    booking = bookings[user_id]
    setattr(booking, property, value)
    missing_values = [k for k, v in booking.dict().items() if v is None]
    missing_values = ", ".join(missing_values)
    return f"Booking ID {user_id} updated with {property} = {value}, missing values {missing_values}"


def create_booking(user_id: str=Field(description="user's id")) -> str:
    """Create a new booking and return the booking ID."""
    bookings[user_id] = Booking()
    return "Booking created, but not yet confirmed. Please provide name, email, phone, date, and time and then use update_booking to update ."


def confirm_booking(user_id: str=Field(description="user's id")) -> str:
    """Confirm a booking for a given booking ID."""
    booking = bookings[user_id]

    if booking.name is None:
        raise ValueError("Please provide your name.")

    if booking.email is None:
        raise ValueError("Please provide your email.")

    if booking.phone is None:
        raise ValueError("Please provide your phone number.")

    if booking.date is None:
        raise ValueError("Please provide the date of your booking.")

    if booking.time is None:
        raise ValueError("Please provide the time of your booking.")

    return f"Booking ID {user_id} confirmed!"



### Controlling Agent Reasoning Loop with Return Direct Tools
modified from https://docs.llamaindex.ai/en/stable/examples/agent/return_direct_agent/

Sometimes, the tool output is good enough. We don't want llm to rewrite the results, we can therefore set `return_direct` to True. This makses tools return outputs directly which greatly speeds up response times. 

In [ ]:

# create tools for each function
get_booking_state_tool = FunctionTool.from_defaults(fn=get_booking_state)
update_booking_tool = FunctionTool.from_defaults(fn=update_booking)
create_booking_tool = FunctionTool.from_defaults(
    fn=create_booking, return_direct=True
)
confirm_booking_tool = FunctionTool.from_defaults(
    fn=confirm_booking, return_direct=True
)

tools = [
        get_booking_state_tool,
        update_booking_tool,
        create_booking_tool,
        confirm_booking_tool,
        ]

In [ ]:
from llama_index.llms.anthropic import Anthropic
from llama_index.core.llms import ChatMessage
from llama_index.core.agent import FunctionCallingAgentWorker

llm = Anthropic(model="claude-3-sonnet-20240229", temperature=0.1)

user = "user123"
prefix_messages = [
    ChatMessage(
        role="system",
        content=(
            f"You are now connected to the booking system and helping {user} with making a booking. "
            "Only enter details that the user has explicitly provided. "
            "Do not make up any details."
        ),
    )
]

worker = FunctionCallingAgentWorker(
    tools=tools,
    llm=llm,
    prefix_messages=prefix_messages,
    max_function_calls=10,
    allow_parallel_tool_calls=False,
    verbose=True,
)

agent = worker.as_agent()

In [6]:
messages = ["Hello! I would like to make a booking, around 5pm?", 
           "Sure! My name is Logan, and my email is test@gmail.com",
           "Right! My phone number is 1234567890, the date of the booking is April 5, at 5pm."]

In [ ]:
for msg in messages:
    response = agent.chat(msg)
    print(msg)
    print(str(response))

In [ ]:
# Check if the user made a booking successfully
print(bookings["user123"])

### Custom Agent
FunctionCallingAgentWorker does not yet support local LLMs. To address this, a workaround using ReActAgent is demonstrated as follows.

In [3]:
# !pip install llama-index-llms-ollama
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3.1", base_url='http://localhost:11434', temperature=0.0, request_timeout=600)

In [87]:
from llama_index.core.agent import ReActAgent
from llama_index.core.llms import ChatMessage

react_agent = ReActAgent.from_tools(tools=tools, llm=llm, verbose=True)

In [88]:
user = 'user123'
task_prefix = f"""You are now connected to the booking system and helping user, id: {user}, with making a booking. 
            Only enter details that the user has explicitly provided. 
            Do not make up any details. Here is the task from the user"""

proxy_prefix = f"""You are now helping user to interact with the booking agent
            If the agent tell you further information is required, you can ask user for the information. 
            Do not tell user you are passing agent's message to the user.
            Do not make up any details. Here is the input from the user:"""

task_str = "Hello! I would like to make a booking, around 5pm?"

def customer_support():
    # Initialize the chat
    first_msg = input("Hello! How can I assist you today?")
    # ReAct agent will use tool best for the task and return the result of the tool(observation)
    agent_response = react_agent.chat(task_prefix +first_msg)
    proxy_str_1 = proxy_prefix + first_msg + "\n\nHere is the response from the agent:" + agent_response.response
    proxy_response = llm.complete(proxy_str_1)
    # Engage with the user to gather more information based on the agent's output.
    ask_user =proxy_response.text
    while True:
        # repeat the process until the user says 'exit'
        usef_msg = input(ask_user)
        if usef_msg=='exit':
            return
        try:
            agent_response = react_agent.chat(usef_msg)
        except:
            print(agent_response)
        proxy_response = llm.complete(proxy_prefix + usef_msg + "\n\nHere is the response from the agent:" + agent_response.response)
        ask_user =proxy_response.text

    

In [89]:
# Dialog example:
# AI: Hello! How can I assist you today?
# YOU: Hello! I would like to make a booking, around 5pm?
# AI: What is your fullname?
# YOU: Amy Brian
# AI: What is your email?
# YOU: amyb@gmail.com
# AI: Thank you Amy! I have made a booking for you at 5pm today.

def customer_support():
    print("AI: Hello! How can I assist you today?")
    user_input = input("YOU: ")
    if "booking" in user_input:
        print("AI: What is your name?")
        name = input("YOU: ")
        print("AI: What is your email?")
        email = input("YOU: ")
        print(f"AI: Thank you {name}! I have made a booking for you at 5pm today. You will receive a confirmation email shortly.")
customer_support()

> Running step 71d10b93-e021-4a97-a0fc-e4ef45339402. Step input: You are now connected to the booking system and helping user, id: user123, with making a booking. 
            Only enter details that the user has explicitly provided. 
            Do not make up any details. Here is the task from the userHello! I would like to make a booking, around 5pm?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: create_booking
Action Input: {'user_id': 'user123', 'property': 'date', 'value': 'around 5pm'}
Observation: Error: create_booking() got an unexpected keyword argument 'property'
> Running step 20236085-7162-451d-8684-1bd5e6928289. Step input: None
Thought: The current language of the user is: English. I need to use a tool that matches the expected arguments.
Action: create_booking
Action Input: {'user_id': 'user123'}
Observation: Booking created, but not yet confirmed. Please provide name, email, phone, date, and time and 

In [90]:
# booking confirmed!
bookings['user123']

Booking(name='Amy Brian', email='amyb@gmail.com', phone='312 1234567', date='today', time='5pm')